### Import Modules

In [1]:
import fiona
from shapely.geometry import shape, Point, asShape
from shapely import speedups
speedups.enable()
import datetime
import pandas

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


### Set paths

In [2]:
points = "/users/danielcorcoran/desktop/test_coordinates_10K.csv"
shapefile = "/users/danielcorcoran/desktop/github_repos/python_nb_data_spatial/data/LGA_ABS_16/LGA_2016_AUST.shp"

### Import points

In [3]:
data = pandas.read_csv("/users/danielcorcoran/desktop/test_coordinates_10K.csv")

In [4]:
data.head()

,longitude,latitude,id
0,144.200311,-36.814039,unique_0
1,144.296159,-32.650504,unique_1
2,147.628548,-38.126470,unique_2
3,144.437691,-37.149164,unique_3
4,147.175428,-37.748786,unique_4


In [5]:
all_polygons = [polygon for polygon in fiona.open(shapefile)]

In [6]:
vic_polygons = []

for polygon in all_polygons:
    if polygon['properties']['STE_NAME16'] == 'Victoria':
        vic_polygons.append(polygon)

In [7]:
print(len(vic_polygons))

82


### Build spatial tree

In [8]:
from rtree import index

In [9]:
rtree_index = index.Index()

In [10]:
for position, polygon in enumerate(vic_polygons):
    try:
        rtree_index.insert(position, shape(polygon['geometry']).bounds)
    except:
        print("Error at position", position)

Error at position 80
Error at position 81


### Process 

In [ ]:
start = datetime.datetime.now()


for index in range(data.shape[0]):
    
    point = Point(data.loc[index, "longitude"], data.loc[index, "latitude"])
      
    for j in rtree_index.intersection(point.coords[0]):
        
        if point.within(shape(vic_polygons[j]['geometry'])):
            for key in vic_polygons[j]['properties'].keys():
                data.loc[index, 'SCRIPT_' + key] = vic_polygons[j]['properties'][key]
            break
            
print("Finished in {}".format(datetime.datetime.now() - start))

In [ ]:
data